In [ ]:
from typing import Optional, Dict, Any, Type, Callable
import logging
import asyncio
from datetime import datetime
from dataclasses import dataclass
from enum import Enum
import traceback
import modelcontextprotocol as mcp

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Error hierarchy
class MCPError(Exception):
    """Base class for all MCP errors."""
    def __init__(self, message: str, details: Optional[Dict[str, Any]] = None):
        super().__init__(message)
        self.message = message
        self.details = details or {}
        self.timestamp = datetime.now()
        
    def __str__(self) -> str:
        return f"{self.__class__.__name__}: {self.message}"

class ToolError(MCPError):
    """Errors related to tool execution."""
    pass

class ResourceError(MCPError):
    """Errors related to resource management."""
    pass

class ProtocolError(MCPError):
    """Errors related to the MCP protocol."""
    pass

class ValidationError(MCPError):
    """Errors related to input validation."""
    pass

# Error severity
class ErrorSeverity(Enum):
    INFO = "info"
    WARNING = "warning"
    ERROR = "error"
    CRITICAL = "critical"

# Error context
@dataclass
class ErrorContext:
    error: Exception
    severity: ErrorSeverity
    tool_name: Optional[str] = None
    operation: Optional[str] = None
    timestamp: datetime = datetime.now()
    stack_trace: str = ""
    
    def __post_init__(self):
        self.stack_trace = "".join(traceback.format_tb(self.error.__traceback__))

# Error handler
class ErrorHandler:
    def __init__(self):
        self.handlers: Dict[Type[Exception], Callable] = {}
        
    def register(self, error_type: Type[Exception], handler: Callable):
        """Register an error handler for a specific error type."""
        self.handlers[error_type] = handler
        
    async def handle(self, context: ErrorContext) -> None:
        """Handle an error using registered handlers."""
        error_type = type(context.error)
        
        # Find the most specific handler
        handler = None
        for err_type, h in self.handlers.items():
            if isinstance(context.error, err_type):
                handler = h
                break
                
        if handler:
            try:
                await handler(context)
            except Exception as e:
                logger.error(f"Error in error handler: {e}")
        else:
            # Default handling
            logger.error(f"Unhandled error: {context}")
            
    def wrap(self, error_type: Type[Exception], severity: ErrorSeverity):
        """Decorator to wrap a function with error handling."""
        def decorator(func):
            async def wrapper(*args, **kwargs):
                try:
                    return await func(*args, **kwargs)
                except Exception as e:
                    if not isinstance(e, error_type):
                        e = error_type(str(e))
                    context = ErrorContext(
                        error=e,
                        severity=severity,
                        operation=func.__name__
                    )
                    await self.handle(context)
                    raise
            return wrapper
        return decorator

# Create global error handler
error_handler = ErrorHandler()


In [ ]:
# Example error handlers
async def log_error(context: ErrorContext):
    """Log error details."""
    logger.error(
        f"Error in {context.operation}: {context.error}\n"
        f"Severity: {context.severity.value}\n"
        f"Stack trace:\n{context.stack_trace}"
    )

async def notify_critical(context: ErrorContext):
    """Simulate sending notifications for critical errors."""
    if context.severity == ErrorSeverity.CRITICAL:
        logger.critical(f"🚨 CRITICAL ERROR: {context.error}")
        # In real implementation: send email, Slack message, etc.

# Register handlers
error_handler.register(MCPError, log_error)
error_handler.register(MCPError, notify_critical)

# Example tool with error handling
class Calculator:
    """A simple calculator tool with error handling."""
    
    class CalculationError(ToolError):
        """Error during calculation."""
        pass
        
    class ValidationError(ValidationError):
        """Invalid input error."""
        pass
    
    @error_handler.wrap(ValidationError, ErrorSeverity.WARNING)
    async def validate_input(self, x: float, y: float) -> None:
        """Validate input values."""
        if not isinstance(x, (int, float)) or not isinstance(y, (int, float)):
            raise self.ValidationError("Inputs must be numbers")
    
    @error_handler.wrap(CalculationError, ErrorSeverity.ERROR)
    async def divide(self, x: float, y: float) -> float:
        """Divide x by y with error handling."""
        await self.validate_input(x, y)
        
        if y == 0:
            raise self.CalculationError(
                "Division by zero",
                details={"x": x, "y": y}
            )
            
        return x / y
    
    @error_handler.wrap(CalculationError, ErrorSeverity.CRITICAL)
    async def complex_calculation(self, x: float, y: float) -> float:
        """Perform a complex calculation that might fail."""
        await self.validate_input(x, y)
        
        try:
            result = x ** y
            if result > 1e308:  # Max float value
                raise OverflowError("Result too large")
            return result
        except OverflowError as e:
            raise self.CalculationError(
                str(e),
                details={"x": x, "y": y}
            )

# MCP models
class CalculationRequest(BaseModel):
    operation: str = Field(..., description="Operation to perform (divide, power)")
    x: float = Field(..., description="First number")
    y: float = Field(..., description="Second number")

class CalculationResult(BaseModel):
    result: float = Field(..., description="Calculation result")

# Create calculator tool
calculator = Calculator()

# Test error handling
async def test_error_handling():
    try:
        # Test division by zero
        print("Testing division by zero...")
        await calculator.divide(10, 0)
    except Calculator.CalculationError as e:
        print(f"Caught expected error: {e}\n")
        
    try:
        # Test invalid input
        print("Testing invalid input...")
        await calculator.divide("10", 2)
    except Calculator.ValidationError as e:
        print(f"Caught expected error: {e}\n")
        
    try:
        # Test overflow
        print("Testing overflow...")
        await calculator.complex_calculation(10, 1000)
    except Calculator.CalculationError as e:
        print(f"Caught expected error: {e}\n")
        
    # Test successful calculation
    print("Testing successful calculation...")
    result = await calculator.divide(10, 2)
    print(f"10 / 2 = {result}")

# Run tests
await test_error_handling()
